# Catenable Deques

### Overview
|         instance         | persistence | amortization | empty | isEmpty |  cons  |  snoc  |     ++     | head |  tail | last |  init |
|:------------------------:|:-----------:|:------------:|:-----:|:-------:|:------:|:------:|:----------:|:----:|:-----:|:----:|:-----:|
|  Simple Catenable Deque  |  persistent |      yes     |  O(1) |   O(1)  | O(1)** | O(1)** | O(log(n))* | O(1) | O(1)* | O(1) | O(1)* |

** amortized time*

*** amortized or worst-case time (depends on the underlying deque instance)*

Note that both instances assume that there is an underlying deque which supports all operations in $O(1)$ amortized or worst-case time.

In [ ]:
class Deque q where

    -- | Construct new (empty) deque
    empty :: q a
    
    -- | Check whether a deque is empty
    isEmpty :: q a -> Bool
    
    -- Insert, inspect and remove the front element
    cons :: a -> q a -> q a
    head :: q a -> a
    tail :: q a -> q a
    
    -- Insert, inspect and remove the rear element
    snoc :: q a -> a -> q a
    last :: q a -> a
    init :: q a -> q a

class Deque d => CatenableDeque d where

    -- | Concatenate two deques together
    (++) :: d a -> d a -> d a

## Simple Catenable Deque
`SimpleCatDeque` is a *structural abstraction* of standard `Deque`s that implements the *implicit recursive slowdown* to provide a concatenation operator that runs in $O(log(n))$ amortized time.

In the implementation and discussion below we assume that there is an existing `Deque` instance that with $O(1)$ operations (either worst-case or amortized). For example both `BankersDeque` and `RealTimeDeque` satisfy this assumption.

### Representation
`SimpleCatDeque d` structurally abstracts a *primitive* `Deque d` and as such forms a non-uniformly recursive structure. Moreover, since it follows the *implicit recursive slowdown*, this deque has two type constructors:
 1. `Shallow` simply holds a primitive deque `d`
 1. `Deep` has three parts: the front - a primitive deque, middle - `SimpleCatDeque` with `d` as elements and rear - also a primitive deque
 
There is an addition invariant imposed on the `Deep` deque which states that both the front and rear primitive deques must contain at least 2 element (otherwise the deque must be downgraded to a `Shallow` one).

### Operations
Starting with the set of simple operations: `cons`, `snoc`, `head` and `last` all simply propagate to the respective operation on the underlying primitive `Deque`. That is either from the `Shallow` type or in case of a `Deep` deque according to the operation to the front or rear part.

`head` and `last` take $O(1)$ steps in the worst-case and depending on the instance of the primitive deque, `cons` with `snoc` run in $O(1)$ worst-case or amortized time.

Next two operations, `tail` and `init`, are symmetric in implementation and just operate in $O(1)$ amortized time on either the front or the rear part of either a deque. In case of a `Shallow`, the operation simply propagates to the primitive deque but for a `Deep` deque there are three cases to consider:
 - `tail f` (`init r`) has at lest than 2 elements => new `Deep` can safely be constructed with that part in place
 - there is not enough elements and the middle is empty => the deque downgrades into a `Shallow` one with the remaining elements transferred to the appropriate side of the rest (using either `dappendL` or `dappendR`)
 - the result is too small and the middle is non-empty => appropriate side (primitive deque) is taken from the middle and the remaining elements are transferred to it as in the previous case forming a new front or rear of new `Deep` deque

Finally, the concatenation operator `++` can be proved to run in $O(log(n))$ amortized time and takes an advantage of the `Deque` operations described above. There are 4 cases depending on the operand types involved in `++` but two of them are symmetric:
 - both deques are `Shallow` => if either one is too small (has less than 2 elements) then it's appended to the appropriate side of the other and the result is `Shallow`, otherwise both operands fill the sides of a new `Deep` deque
 - one of the operands is `Shallow` and the other is `Deep` => if the shallow deque is too small then its element is moved to the appropriate side of the deep one, otherwise it replaces such side and the original front/rear is moved to the middle part
 - both deques are `Deep` => front of the former and rear of the latter deque form new front and rear and new middle is constructed by concatenating the rest of the first operand with the initial part of the second operand

Last but not least, the complexity of `++` is indeed logarithmic but it can be stated more precisely with respect to the sizes $n_1$ and $n_2$ of the first and second operand respectively as $O(min(log(n_1), log(n_2)))$.

In [ ]:
data SimpleCatDeque d a = Shallow (d a) | Deep (d a) (SimpleCatDeque d (d a)) (d a)

-- | Determines whether given deque 'd' contains less than 2 elements (i.e. is empty or singleton).
-- |
-- | Note: Deque 'd' is assumed to provide 'tail' in O(1) worst-case or amortized time.
tooSmall :: Deque d => d a -> Bool
tooSmall d = isEmpty d || isEmpty (tail d)

-- | Rotate the head of the first deque to the front of the second one.
-- |
-- | Note: Deque 'd' is assumed to provide 'cons' and 'head' in O(1) worst-case or amortized time.
dappendL :: Deque d => d a -> d a -> d a
dappendL d1 d2 = if isEmpty d1 then d2 else cons (head d1) d2

-- | Rotate the head of the second deque to the back of the first one.
-- |
-- | Note: Deque 'd' is assumed to provide 'snoc' and 'head' in O(1) worst-case or amortized time.
dappendR :: Deque d => d a -> d a -> d a
dappendR d1 d2 = if isEmpty d2 then d1 else snoc d1 (head d2)

-- | 'Deque' instance for 'SimpleCatDeque d' given an arbitrary deque 'd'.
-- |
-- | Assumption: Deque 'd' supports all operations in O(1) time (either amortized or worst-case).
instance Deque d => Deque (SimpleCatDeque d) where
    
    -- | Construct new (empty) deque.
    empty = Shallow empty
    
    -- | Check whether a deque is empty.
    isEmpty (Shallow d) = isEmpty d
    isEmpty _ = False
    
    -- | Prepend an element to the front in O(1) worst-case or amortized time (depends on 'd').
    cons x (Shallow d) = Shallow (cons x d)
    cons x (Deep f m r) = Deep (cons x f) m r
    
    -- | Peek the front element of the deque in O(1) worst case time.
    head (Shallow d) = head d
    head (Deep f _ _) = head f
    
    -- | Remove the front element from a non-empty deque in O(1) amortized time.
    -- |
    -- | Note: By the assumtion on 'd', 'dappendL' runs in constant time (amortized or worst-case).
    tail (Shallow d) = Shallow (tail d)
    tail (Deep f m r)
            | not $ tooSmall f' = Deep f' m r
            | isEmpty m = Shallow (dappendL f' r)
            | otherwise = Deep (dappendL f' (head m)) (tail m) r
        where f' = tail f
    
    -- | Appends an element to the back in O(1) worst-case or amortized time (depends on 'd').
    snoc (Shallow d) x = Shallow (snoc d x)
    snoc (Deep f m r) x = Deep f m (snoc r x)
    
    -- | Peek the rear element of the deque in O(1) worst case time.
    last (Shallow d) = last d
    last (Deep _ _ r) = last r
    
    -- | Remove the rear element from a non-empty deque in O(1) amortized time.
    -- |
    -- | Note: By the assumtion on 'd', 'dappendR' runs in constant time (amortized or worst-case).
    init (Shallow d) = Shallow (init d)
    init (Deep f m r)
            | not $ tooSmall r' = Deep f m r'
            | isEmpty m = Shallow (dappendR f r')
            | otherwise = Deep f (init m) (dappendR (last m) r')
        where r' = init r

instance Deque d => CatenableDeque (SimpleCatDeque d) where
    
    -- | Concatenate two deques together in O(log(n)) amortized time.
    (Shallow d1) ++ (Shallow d2)
        | tooSmall d1 = Shallow (dappendL d1 d2)
        | tooSmall d2 = Shallow (dappendR d1 d2)
        | otherwise = Deep d1 empty d2
    (Shallow d) ++ (Deep f m r)
        | tooSmall d = Deep (dappendL d f) m r
        | otherwise = Deep d (cons f m) r
    (Deep f m r) ++ (Shallow d)
        | tooSmall d = Deep f m (dappendR r d)
        | otherwise = Deep f (snoc m r) d
    (Deep f1 m1 r1) ++ (Deep f2 m2 r2) = Deep f1 (snoc m1 r1 ++ cons f2 m2) r2